# Building a Memory-Enhanced Conversational Agent

## Overview
This tutorial outlines the process of creating a conversational AI agent with enhanced memory capabilities. The agent incorporates both short-term and long-term memory to maintain context and improve the quality of interactions over time.

## Motivation
Traditional chatbots often struggle with maintaining context beyond immediate interactions. This limitation can lead to disjointed conversations and a lack of personalization. By implementing both short-term and long-term memory, we aim to create an agent that can:
- Maintain context within a single conversation
- Remember important information across multiple sessions
- Provide more coherent and personalized responses

## Key Components
1. **Language Model**: The core AI component for understanding and generating responses.
2. **Short-term Memory**: Stores the immediate conversation history.
3. **Long-term Memory**: Retains important information across multiple conversations.
4. **Prompt Template**: Structures the input for the language model, incorporating both types of memory.
5. **Memory Manager**: Handles the storage and retrieval of information in both memory types.

## Method Details

### Setting Up the Environment
1. Import necessary libraries for the language model, memory management, and prompt handling.
2. Initialize the language model with desired parameters (e.g., model type, token limit).

### Implementing Memory Systems
1. Create a store for short-term memory (conversation history):
   - Use a dictionary to manage multiple conversation sessions.
   - Implement a function to retrieve or create new conversation histories.

2. Develop a long-term memory system:
   - Create a separate store for persistent information.
   - Implement functions to update and retrieve long-term memories.
   - Define simple criteria for what information to store long-term (e.g., longer user inputs).

### Building the Conversational Logic
1. Combine the prompt template with the language model.
2. Wrap this combination with a component that manages message history.
3. Ensure the chain can access and update both short-term and long-term memory.

### Creating the Interaction Loop
1. Develop a main chat function that:
   - Retrieves relevant long-term memories.
   - Invokes the conversational chain with the current input and memories.
   - Updates the long-term memory based on the interaction.
   - Returns the AI's response.

### Testing and Refinement
1. Run example conversations to test the agent's ability to maintain context.
2. Review both short-term and long-term memories after interactions.
3. Adjust memory management criteria and prompt structure as needed.

## Conclusion
The Memory-Enhanced Conversational Agent offers several advantages over traditional chatbots:

- **Improved Context Awareness**: By utilizing both short-term and long-term memory, the agent can maintain context within and across conversations.
- **Personalization**: Long-term memory allows the agent to remember user preferences and past interactions, enabling more personalized responses.
- **Flexible Memory Management**: The implementation allows for easy adjustment of what information is stored long-term and how it's used in conversations.
- **Scalability**: The session-based approach allows for managing multiple independent conversations.

This implementation provides a foundation for creating more sophisticated AI agents. Future enhancements could include:
- More advanced criteria for long-term memory storage
- Implementation of memory consolidation or summarization techniques
- Integration with external knowledge bases
- Emotional or sentiment tracking across interactions

By focusing on memory enhancement, this conversational agent design significantly improves upon basic chatbot functionality, paving the way for more engaging, context-aware, and intelligent AI assistants.

## Setup and Imports

First, we'll import the necessary modules and set up our environment.

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_base_url = os.getenv("OPENAI_BASE_URL")

# Initialize OpenAI client
client = OpenAI()

# Initialize the language model
model = "gpt-4o-mini"

## Memory Stores

We'll create stores for both short-term (chat history) and long-term memory.

In [2]:
chat_store = {}  # Store short-term chat history for each session
long_term_memory = {}  # Store long-term memory for each session
# e.g. long_term_memory = {"session_id": [{"role": "user", "content": "Hello"}, {"role": "assistant", "content": "Hi there!"}]}"}

# Function to get chat history
def get_chat_history(session_id: str):
    if session_id not in chat_store:
        chat_store[session_id] = []
    return chat_store[session_id]

# Function to update long-term memory
def update_long_term_memory(session_id: str, input_text: str, output_text: str):
    if session_id not in long_term_memory:
        long_term_memory[session_id] = []
    
    if len(input_text) > 20:
        message = [
            {"role": "user", "content": input_text},
            {"role": "assistant", "content": output_text}
        ]
        long_term_memory[session_id].append(message)
    
    if len(long_term_memory[session_id]) > 5:
        long_term_memory[session_id] = long_term_memory[session_id][-5:]

# Function to get long-term memory
def get_long_term_memory(session_id: str):
    return ". ".join(str(item) for item in long_term_memory.get(session_id, []))

## Response Generation

To generate responses, we manually build the prompt and call the API.

In [3]:
def generate_response(prompt: str):
    response = client.chat.completions.create(
        model=model ,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,  # Controls randomness
        max_tokens=150  # Limits the number of tokens in the response
    )
    return response.choices[0].message.content

## Conversational Logic

This is the core of the conversation flow, where we combine short-term and long-term memories and generate the response.

In [4]:
def chat(input_text: str, session_id: str):
    long_term_mem = get_long_term_memory(session_id)
    chat_history = get_chat_history(session_id)
    
    # Build the prompt, including both long-term memory and chat history
    prompt = f"You are a helpful AI assistant. Use the information from long-term memory if relevant.\n"
    prompt += f"Long-term memory: {long_term_mem}\n"
    messages = [
        {"role": "system", "content": prompt}
    ]
    for message in chat_history:
        messages.append({"role": message["role"], "content": message["content"]})
    messages.append({"role": "user", "content": input_text})
    
    response = generate_response(prompt)
    
    # Add the user input and AI response to the chat history
    chat_history.append({"role": "user", "content": input_text})
    chat_history.append({"role": "ai", "content": response})
    
    # Update long-term memory
    update_long_term_memory(session_id, input_text, response)
    
    return response


## Example Usage

Let's test our memory-enhanced conversational agent with a series of interactions.

In [5]:
session_id = "user_123"

print("AI:", chat("Hello! My name is Alice.", session_id))
print("AI:", chat("What's the weather like today?", session_id))
print("AI:", chat("I love sunny days.", session_id))
print("AI:", chat("Do you remember my name?", session_id))

AI: It seems like you might want to provide some specific information or context related to long-term memory. Please share what you would like to discuss or inquire about, and I'll do my best to assist you!
AI: Hello, Alice! How can I assist you today? If there's something specific you'd like to discuss or ask about, feel free to let me know!
AI: Hello, Alice! How can I assist you today? If you have a specific question or topic in mind, just let me know!
AI: Hello, Alice! I hope you're doing well. How can I assist you today? If you have any questions or topics in mind, just let me know!


## Review Memory

Let's review the conversation history and long-term memory.

In [6]:
print("\nConversation History:")
for message in chat_store[session_id]:
    print(f"{message['role']}: {message['content']}")

print("\nLong-term Memory:")
print(get_long_term_memory(session_id))


Conversation History:
user: Hello! My name is Alice.
ai: It seems like you might want to provide some specific information or context related to long-term memory. Please share what you would like to discuss or inquire about, and I'll do my best to assist you!
user: What's the weather like today?
ai: Hello, Alice! How can I assist you today? If there's something specific you'd like to discuss or ask about, feel free to let me know!
user: I love sunny days.
ai: Hello, Alice! How can I assist you today? If you have a specific question or topic in mind, just let me know!
user: Do you remember my name?
ai: Hello, Alice! I hope you're doing well. How can I assist you today? If you have any questions or topics in mind, just let me know!

Long-term Memory:
[{'role': 'user', 'content': 'Hello! My name is Alice.'}, {'role': 'assistant', 'content': "It seems like you might want to provide some specific information or context related to long-term memory. Please share what you would like to discus